# TensorFlow with Estimators

In this note, we use iris dataset to learn TensorFlow Estimators.

## Estimator Steps

1. Read in data (+ normalize)
2. Train/test split
3. Create feature columns
4. Create estimator input function and estimator model
5. Train estimator model
6. Predict with test input function
7. Evaluation

## Read in Data

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('iris.csv')

In [4]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


To use TensorFlow estimator, there are some rules:
1. the dataset's column name can NOT include special characters (ex. '(' )
2. the target should be integer.

Change the column name:

In [5]:
df.columns = ['sepal_length','sepal_width','petal_length','petal_width','target']

Spilt dataset into X and y, where y applied int casting:

In [6]:
X = df.drop('target',axis=1)
y = df['target'].apply(int)

## Train Test Split

In [59]:
from sklearn.model_selection import train_test_split

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Create Feature Columns

Reference: [Feature Columns](https://www.tensorflow.org/guide/feature_columns)

{% asset_img feature_columns_1.jpg %}
{% asset_img feature_columns_2.jpg %}

In [61]:
import tensorflow as tf

In [62]:
X.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object')

Transfer X columns from Pandas Series to TensorFlow Feature Columns(Numerical):

In [63]:
feat_cols = []

for col in X.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [64]:
type(X['sepal_length'])

pandas.core.series.Series

In [65]:
feat_cols

[NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Create Estimator Input Function and Estimator Model

### Create Estimator Input Function:

In [66]:
# there is also a pandas_input_fn we'll see in the exercise!!
# num_epochs: limitation of training times. 
#             num_epochs= 5 means the training will be stopped after go through every single training point for 5 times.           
# shuffle: if our target is sorted, we can use shuffle=True to shuffle it around.
#    
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=5,shuffle=True)

### Create DNN Classifier

To create DNN classifier, we need to know how many classes there are for the iris:

In [67]:
set(y)

{0, 1, 2}

It means we have 3 species: {0, 1, 2}

In [68]:
# hidden_units: list of number of neurons in each layer. len(hidden_units) = number of hidden layers
#               hidden_units=[10, 20, 10]=> layers = input + 10 + 20 + 10 + output layer
# n_classes: how many classes we expect 
classifier = tf.estimator.DNNClassifier(hidden_units=[10, 20, 10], n_classes=3,feature_columns=feat_cols)

W0830 07:30:31.921795  3244 estimator.py:1811] Using temporary folder as model directory: C:\Users\User\AppData\Local\Temp\tmptv_dhs6t


## Train Estimator Model

In [69]:
classifier.train(input_fn=input_func,steps=50)

## Prediction with Test Input Function

** Use the predict method from the classifier model to create predictions from X_test **

Create the input function for prediction:

In [70]:
pred_input_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

Use classifier to predict pred_input_fn:

In [71]:
predictions = list(classifier.predict(input_fn=pred_input_fn))

In [72]:
predictions[0]

{'logits': array([-2.4918616,  1.0184311,  0.2909072], dtype=float32),
 'probabilities': array([0.01975435, 0.6609421 , 0.3193036 ], dtype=float32),
 'class_ids': array([1], dtype=int64),
 'classes': array([b'1'], dtype=object),
 'all_class_ids': array([0, 1, 2]),
 'all_classes': array([b'0', b'1', b'2'], dtype=object)}

In [73]:
y_pred  = []
for pred in predictions:
    y_pred.append(pred['class_ids'][0])

## Evaluation

Here we use [Classfication Report](https://medium.com/@chih.sheng.huang821/%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92-%E7%B5%B1%E8%A8%88%E6%96%B9%E6%B3%95-%E6%A8%A1%E5%9E%8B%E8%A9%95%E4%BC%B0-%E9%A9%97%E8%AD%89%E6%8C%87%E6%A8%99-b03825ff0814) and [Confusion Matrix](https://www.ycc.idv.tw/confusion-matrix.html) for Evaluation:

In [74]:
from sklearn.metrics import classification_report,confusion_matrix

In [75]:
print(confusion_matrix(y_test,y_pred))

[[16  0  0]
 [ 0 14  2]
 [ 0  0 13]]


In [76]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.88      0.93        16
           2       0.87      1.00      0.93        13

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.95        45
weighted avg       0.96      0.96      0.96        45



## Reference

[Python for Data Science and Machine Learning Bootcamp](https://www.udemy.com/python-for-data-science-and-machine-learning-bootcamp/)

[Feature Columns - TensorFlow Core](https://www.tensorflow.org/guide/feature_columns)

[如何辨別機器學習模型的好壞？秒懂Confusion Matrix](https://www.ycc.idv.tw/confusion-matrix.html)

[機器學習\統計方法: 模型評估-驗證指標(validation index)](https://medium.com/@chih.sheng.huang821/%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92-%E7%B5%B1%E8%A8%88%E6%96%B9%E6%B3%95-%E6%A8%A1%E5%9E%8B%E8%A9%95%E4%BC%B0-%E9%A9%97%E8%AD%89%E6%8C%87%E6%A8%99-b03825ff0814)

